In [1]:
from __future__ import division, print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [2]:
#this is set up for 1d convolutions where examples
#have dimensions (len, num_channels)
#the channel axis is the axis for one-hot encoding.
def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1

In [3]:
import pyfasta
import gzip
import numpy as np

file_with_regions = "1kb_around_summit_H1-hESC-NANOG-human-ENCSR000BMT-optimal_idr.bed"
test_chromosomes = ["chr1"]

fasta_data_source = "/mnt/data/annotations/by_organism/human/hg19.GRCh37/hg19.genome.fa"
f = pyfasta.Fasta(fasta_data_source)

fasta_sequences = []

for line in gzip.open(file_with_regions, "rb"):
    line = line.rstrip()
    chrom,start,end = line.split("\t")
    start,end = int(start),int(end)
    the_seq = f[chrom][start:end]
    fasta_sequences.append(the_seq)

onehot_fasta = np.array([one_hot_encode_along_channel_axis(x) for x in fasta_sequences])
print(onehot_fasta.shape)


(8307, 1000, 4)


In [4]:
import os
import keras
from keras.models import model_from_json

model_h5 = "record_1_model_HcVec_modelWeights.h5"
model_json = "record_1_model_HcVec_modelJson.json"
the_model = model_from_json(open(model_json).read())
the_model.load_weights(model_h5)

Using TensorFlow backend.
/users/avanti/anaconda2/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/users/avanti/anaconda2/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/users/avanti/anaconda2/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/users/avanti/anaconda2/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \


In [5]:
import deeplift
from deeplift.conversion import kerasapi_conversion as kc

deeplift_model =\
    kc.convert_model_from_saved_files(
        h5_file=model_h5, json_file=model_json) 

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 2 the preceding linear layer is 0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 5 the preceding linear layer is 3 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
For layer 8 the preceding linear layer is 6 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
For layer 13 the preceding linear layer is 11 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
Heads-up: I assume sigmoid is the output layer, not an intermediate one; if it's an intermediate layer then please bug me and I will imple

In [6]:
from deeplift.util import get_hypothetical_contribs_func_onehot
from deeplift.util import get_shuffle_seq_ref_function
from deeplift.dinuc_shuffle import dinuc_shuffle

multipliers_func = deeplift_model.get_target_multipliers_func(
                            find_scores_layer_idx=0)

hypothetical_contribs_func = get_hypothetical_contribs_func_onehot(multipliers_func)
hypothetical_contribs_many_refs_func = get_shuffle_seq_ref_function(
    score_computation_function=hypothetical_contribs_func,
    shuffle_func=dinuc_shuffle,
    one_hot_func=lambda x: np.array([one_hot_encode_along_channel_axis(seq)
                                     for seq in x]))
num_refs_per_seq=10
hypothetical_contribs = hypothetical_contribs_many_refs_func(
            task_idx=0,
            input_data_sequences=fasta_sequences,
            num_refs_per_seq=num_refs_per_seq,
            batch_size=50,
            progress_update=4000,
        )

4000 reference seqs generated
8000 reference seqs generated
12000 reference seqs generated
16000 reference seqs generated
20000 reference seqs generated
24000 reference seqs generated
28000 reference seqs generated
32000 reference seqs generated
36000 reference seqs generated
40000 reference seqs generated
44000 reference seqs generated
48000 reference seqs generated
52000 reference seqs generated
56000 reference seqs generated
60000 reference seqs generated
64000 reference seqs generated
68000 reference seqs generated
72000 reference seqs generated
76000 reference seqs generated
80000 reference seqs generated
One hot encoding sequences...
One hot encoding done...
Done 0
Done 4000
Done 8000
Done 12000
Done 16000
Done 20000
Done 24000
Done 28000
Done 32000
Done 36000
Done 40000
Done 44000
Done 48000
Done 52000
Done 56000
Done 60000
Done 64000
Done 68000
Done 72000
Done 76000
Done 80000


In [ ]:
revcomp = {"A": "T", "C": "G", "G": "C", "T": "A"}
reverse_fasta_sequences = ["".join(revcomp[x] for x in seq)
                           for seq in fasta_sequences]